This iPythonNotebook can be used to experiment with the surgery toolkit, where the user will be able to modify the network with their rules. The modified network can be itself used to further train on the desired dataset. This allows user to easily modify any network.

In [1]:
!pip install netron

import torch
import torch.nn as nn
import edgeai_torchmodelopt
import copy
import netron

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.9 MB/s eta 0:00:0000:0100:01


The required imports are done, netron is installed and used to visualize the network. 

In [2]:
class SimpleNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv0 = nn.Conv2d(3, 32, 3, padding=1, stride=2)
        self.bn0 = nn.BatchNorm2d(32)
        self.relu0 = nn.ReLU()
        
        self.conv1 = nn.Conv2d(32, 32, 3, padding=1, stride=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv0(x)
        x = self.bn0(x)
        x = self.relu0(x)
    
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
    
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
    
        return x
    
model = SimpleNetwork()
example_input = torch.ones((1, 3, 224, 224))

y = model(example_input)
print(model)
print("Output Shape is : {}".format(y.shape))

SimpleNetwork(
  (conv0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
)
Output Shape is : torch.Size([1, 32, 28, 28])


In [3]:
model_export_name = "./orig_simple_network_surgery.onnx"
torch.onnx.export(model, example_input, model_export_name)
netron.start(model_export_name, 8081)

Serving './orig_simple_network_surgery.onnx' at http://localhost:8081


('localhost', 8081)

We can get a default dictionary for optimization based on the most optimal conversions for TIDL. These changes might require re-training of the network. The user can use the flag : can_retrain=False, to only do the modifications which do not require retraining. The replacement dictionary can be defined by the user as well.

In [4]:
replacement_flags = copy.deepcopy(edgeai_torchmodelopt.xmodelopt.surgery.v2.get_replacement_flag_dict_default())
replacement_dict = edgeai_torchmodelopt.xmodelopt.surgery.v2.get_replacement_dict(replacement_flags)
for orig, replace in replacement_dict.items():
    print(str(orig) + " ---> " + str(replace) + "\n")


None ---> <class 'torch.nn.modules.linear.Identity'>

SEModule(
  (sequence): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
    (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
    (4): Hardsigmoid()
  )
) ---> Identity()

SEModule1(
  (sequence): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
    (2): SiLU()
    (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
    (4): Sigmoid()
  )
) ---> Identity()

se_layer ---> <function replace_se_layer at 0x79afd651e560>

<class 'torch.nn.modules.activation.ReLU'> ---> <class 'torch.nn.modules.activation.ReLU'>

<class 'torch.nn.modules.activation.GELU'> ---> <class 'torch.nn.modules.activation.ReLU'>

<class 'torch.nn.modules.activation.ReLU6'> ---> <class 'torch.nn.modules.activation.ReLU'>

<class 'torch.nn.modules.activation.SiLU'> ---> <class 'torch.nn.modules.activation.ReLU'>

<cl

The replacement dictionary can be defined by the user as well. Multiple other methods including functions are supported which can be read more about [over here](../edgeai_torchmodelopt/xmodelopt/surgery/v2/docs/details.md) under the different-types-of-possible-replacement-rules section.
The converted model can be obtained by a simple call to edgeai_torchmodelopt.xmodelopt.surgery.v2.convert_to_lite_fx function.

In [5]:
replacement_dict = ({torch.nn.ReLU: torch.nn.GELU})
converted_model = edgeai_torchmodelopt.xmodelopt.surgery.v2.convert_to_lite_fx(model, replacement_dict=replacement_dict)
print(converted_model)

SimpleNetwork(
  (conv0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
)



def forward(self, x):
    conv0 = self.conv0(x);  x = None
    bn0 = self.bn0(conv0);  conv0 = None
    relu0 = self.relu0(bn0);  bn0 = None
    conv1 = self.conv1(relu0);  relu0 = None
    bn1 = self.bn1(conv1);  conv1 = None
    relu1 = self.relu1(bn1);  bn1 = None
    conv2 = self.conv2(relu1);  relu1 = None
    bn2 = self.bn2(conv2);  conv2 = None
    relu2 = self.relu2(bn2);  bn2 = None
    return relu2
    
# To see mor

/data/ssd/files/a0507161/edgeai/edgeai-modeloptimization/torchmodelopt/edgeai_torchmodelopt/xmodelopt/surgery/v2/__init__.py:159: UserWarning: if <class 'torch.nn.modules.activation.ReLU'> is not a default flag or its value is not a boolean, the value must be a dict. So, this entry will be discarded!
  warnings.warn(f'if {k} is not a default flag or its value is not a boolean, the value must be a dict. So, this entry will be discarded!')


In [6]:
model_export_name = "./converted_simple_network_surgery.onnx"
torch.onnx.export(converted_model, example_input, model_export_name)
netron.start(model_export_name, 8081)

Stopping http://localhost:8081


Serving './converted_simple_network_surgery.onnx' at http://localhost:8081


('localhost', 8081)